In [1]:
# 데이터 처리 라이브러리
import os
import os.path as pth
#from tqdm import tqdm
import pandas as pd 
import numpy as np
#import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
# import cv2

# import mediapipe as mp

# Tensorflow 관련 라이브러리
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, BatchNormalization, Flatten, Activation
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam


# GPU 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # 특정 GPU에 1GB 메모리만 할당하도록 제한
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        
        tf.config.experimental.set_memory_growth(gpus[0], True)
        
#         tf.config.experimental.set_virtual_device_configuration(
#             gpus[0],
#             [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=15000)])
    except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
        print(e)

# 1. TFrecord 불러오기

In [7]:
train_tfrecord_path = pth.join('./data/emotion_img_tf2', 'tf_record_train.tfrecords')
valid_tfrecord_path = pth.join('./data/emotion_img_tf2', 'tf_record_train.tfrecords')

# BUFFER_SIZE, BATCH_SIZE
BUFFER_SIZE = 100
BATCH_SIZE = 30
NUM_CLASS = 8



image_feature_description = {
    'image_raw': tf.io.FixedLenFeature([], tf.string),
    'img_id': tf.io.FixedLenFeature([], tf.string),
    # 'id': tf.io.FixedLenFeature([], tf.string),
}



def _parse_image_function(example_proto):
    return tf.io.parse_single_example(example_proto, image_feature_description)

def map_func(target_record):
    img = target_record['image_raw']
    label = int(target_record['img_id'])
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.dtypes.cast(img, tf.float32)
    return img, label

def prep_func(image, label):
    print(type(label))

    result_image = image / 255
    result_image = tf.image.resize(result_image, (270,480))

    onehot_label = tf.one_hot(label, depth=NUM_CLASS)
    return result_image, onehot_label

    

dataset = tf.data.TFRecordDataset(train_tfrecord_path, compression_type='GZIP')
dataset = dataset.map(_parse_image_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.map(map_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.map(prep_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

valid_dataset = tf.data.TFRecordDataset(valid_tfrecord_path, compression_type='GZIP')
valid_dataset = valid_dataset.map(_parse_image_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.map(map_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.shuffle(BUFFER_SIZE)
valid_dataset = valid_dataset.batch(BATCH_SIZE)
valid_dataset = valid_dataset.map(prep_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>


In [8]:
earlystop = EarlyStopping(patience=5)
learning_rate_reduction = ReduceLROnPlateau(monitor ='val_accuracy',
                                            patience = 2,
                                            factor = 0.5,
                                            min_lr = 1e-7,
                                            verbose = 1)

model_check = ModelCheckpoint(filepath = './model/facial_emotion_cnn_270_480_adam_h5',
                              monitor = 'val_loss',
                              save_best_only = True)

callbacks = [earlystop, learning_rate_reduction, model_check]

# 2. 모델링

## 2-1. Base CNN

In [9]:
with tf.device('/device:GPU:0'):
    
    model = Sequential()
    
    model.add(Conv2D(8, (3, 3), activation='relu', input_shape=(270, 480, 3)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(16, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
#     model.add(Conv2D(32, (3, 3), activation='relu'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(8, activation='softmax'))
    
    model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 268, 478, 8)       224       
_________________________________________________________________
batch_normalization_3 (Batch (None, 268, 478, 8)       32        
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 134, 239, 8)       0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 134, 239, 8)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 132, 237, 16)      1168      
_________________________________________________________________
batch_normalization_4 (Batch (None, 132, 237, 16)      64        
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 66, 118, 16)      

In [10]:
with tf.device('/device:GPU:0'):
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [ ]:
with tf.device('/device:GPU:0'):
    history = model.fit(dataset,
                        epochs=1000,
                        validation_data=valid_dataset,
                        callbacks = callbacks
                       )

Epoch 1/1000
   2516/Unknown - 954s 379ms/step - loss: 0.9215 - accuracy: 0.5530INFO:tensorflow:Assets written to: ./model/facial_emotion_cnn_270_480_adam_h5/assets
2516/2516 [==============================] - 1898s 754ms/step - loss: 0.9215 - accuracy: 0.5530 - val_loss: 3.8878 - val_accuracy: 0.1607 - lr: 0.0010
Epoch 2/1000
2516/2516 [==============================] - 1894s 753ms/step - loss: 1.0625 - accuracy: 0.4585 - val_loss: 2386.1421 - val_accuracy: 0.1299 - lr: 0.0010
Epoch 3/1000
2516/2516 [==============================] - ETA: 0s - loss: 1.5540 - accuracy: 0.2355